In [1]:
import cdsapi
import os
import shutil
from config import Config
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Créez une instance du client CDS
c = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="249316:cfa30517-00b5-4ded-9330-78b7b21349bd")

# Liste des années à récupérer
years = list(range(1961, 2004)) 
# Dictionnaire pour les décennies
decade = {"01": range(1, 11), "02": range(11, 21), "03": range(21, 32)}

# Nombre de threads pour le téléchargement en parallèle
num_threads = 15

area_lef=[5, -40, -5, 15]   #'area': [5, -40, -5, 15], #sst_lef_
area_nino=[5, -120, -5, -170] #'area': [5, -120, -5, -170], #sst_nino_
area_bf=[15.08, -5.51, 9.41, 2.41] #[15.08, -5.51, 9.41, 2.41],

# Créez une liste de toutes les tâches de téléchargement
tasks = [(year, month, dek, days) for year in years for month in range(1,13) for dek, days in decade.items()]

In [3]:
# Fonction de téléchargement des données
def download_data(year, month, dek, days, parametre, area=area_bf, level=None):
    file_prex=os.path.basename(output_directory)
    output_filename = os.path.join(output_directory, f'{file_prex}_{year}_{month:02d}_{dek}.nc')
    
    if not os.path.isfile(output_filename):
        request = {
            'product_type': 'reanalysis',
            'variable': parametre, #'sea_surface_temperature',
            'year': str(year),
            'month': str(month).zfill(2),
            'day': [str(day).zfill(2) for day in days],
            'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
            'format': 'netcdf',
            'area': area,
        }

        try:
            c.retrieve('reanalysis-era5-single-levels', request, output_filename)
            print(f"Téléchargement réussi pour {year}-{month:02d} {dek}")
        except Exception as e:
            print(f"Erreur lors du téléchargement pour {year}-{month:02d} {dek}: {e}")
            

In [4]:
# Fonction de téléchargement des données
def download_data_level(year, month, dek, days, parametre, area=area_bf, level=None):
    file_prex=os.path.basename(output_directory)
    output_filename = os.path.join(output_directory, f'{file_prex}_{year}_{month:02d}_{dek}.nc')
    
    if not os.path.isfile(output_filename):
        request = {
            'product_type': 'reanalysis',
            'variable': parametre, #'sea_surface_temperature',
            'year': str(year),
            'pressure_level': level,
            'month': str(month).zfill(2),
            'day': [str(day).zfill(2) for day in days],
            'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
            'format': 'netcdf',
            'area': area,
        }

        try:
            c.retrieve('reanalysis-era5-pressure-levels', request, output_filename)
            print(f"Téléchargement réussi pour {year}-{month:02d} {dek}")
        except Exception as e:
            print(f"Erreur lors du téléchargement pour {year}-{month:02d} {dek}: {e}")


### I TEMPERATURE MER

##### I-1 TEMPERATURE MER LANGUE 

In [5]:
# Chemin du dossier de sortie
output_directory =Config.SST_LEF_DEK
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data, year, month, dek, days,'sea_surface_temperature',area_lef) for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")

2024-07-15 17:46:42,631 INFO Welcome to the CDS
2024-07-15 17:46:42,639 INFO Welcome to the CDS
2024-07-15 17:46:42,641 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:42,639 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:42,639 INFO Welcome to the CDS
2024-07-15 17:46:42,644 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:42,894 INFO Request is completed
2024-07-15 17:46:42,894 INFO Request is completed
2024-07-15 17:46:42,895 INFO Request is completed
2024-07-15 17:46:42,899 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data0/adaptor.mars.internal-1721064485.1227477-12129-6-2e47b3c5-e932-4b27-b5b8-254bdab411e3.nc to ..\ASSETS\DATA\RAW\ERA5\SST_LEF\SST_LEF_2001_01_01.nc (1.4M)
2024-07-15 17:46:42,895 INFO Downloading https://d

Téléchargement réussi pour 2001-01 03
Téléchargement réussi pour 2001-01 01


2024-07-15 17:46:45,390 INFO Download rate 570K/s   


Téléchargement réussi pour 2001-01 02
Téléchargement terminé.


##### I-2 TEMPERATURE MER NINO 3.4

In [6]:
# Chemin du dossier de sortie
output_directory =Config.SST_NINO_DEK  # 
#if os.path.exists(output_directory):
#    shutil.rmtree(output_directory)
# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data, year, month, dek, days,'sea_surface_temperature',area_nino) for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")

#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:46:45,758 INFO Welcome to the CDS
2024-07-15 17:46:45,758 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:45,804 INFO Welcome to the CDS
2024-07-15 17:46:45,806 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:45,825 INFO Welcome to the CDS
2024-07-15 17:46:45,834 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:46:46,024 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data1/adaptor.mars.internal-1721065175.396839-3442-3-c413be2e-7091-4842-9464-5dcd2e159c5a.nc to ..\ASSETS\DATA\RAW\ERA5\SST_NINO\SST_NINO_2001_01_01.nc (7.8M)
2024-07-15 17:46:46,034 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data4/adaptor.mars.internal-1720920842.168267-18112-14-2e9b0c3c-5cc6-46e9-a661-efe8d

Téléchargement réussi pour 2001-01 01











2024-07-15 17:46:49,334 INFO Download rate 2.6M/s




Téléchargement réussi pour 2001-01 03










































































































2024-07-15 17:47:11,105 INFO Download rate 318.1K/s


Téléchargement réussi pour 2001-01 02
Téléchargement terminé.


### II POINT DE ROSEE

In [7]:
# Chemin du dossier de sortie
output_directory =Config.TEMP_ROS_DEK
#if os.path.exists(output_directory):
#    shutil.rmtree(output_directory)
# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data, year, month, dek, days,'2m_dewpoint_temperature',area_bf) for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")


#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:47:11,483 INFO Welcome to the CDS
2024-07-15 17:47:11,499 INFO Welcome to the CDS
2024-07-15 17:47:11,499 INFO Welcome to the CDS
2024-07-15 17:47:11,499 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:11,499 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:11,499 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:11,767 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data9/adaptor.mars.internal-1720923797.4465582-8876-14-d72f9962-97d8-42f8-93d7-556c14c2fd92.nc to ..\ASSETS\DATA\RAW\ERA5\TEMP_ROS\TEMP_ROS_2001_01_02.nc (116.6K)
2024-07-15 17:47:11,814 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data0/adaptor.mars.internal-1720923763.4274533-5247-16-da628d4b-55fd-41f9-8245-2

Téléchargement réussi pour 2001-01 01


 79%|███████▉  | 92.0k/117k [00:00<00:00, 267kB/s]2024-07-15 17:47:12,834 INFO Download rate 110.9K/s
2024-07-15 17:47:12,906 INFO Download rate 117.3K/s


Téléchargement réussi pour 2001-01 02
Téléchargement réussi pour 2001-01 03
Téléchargement terminé.


### III VOLUMETRIQUE SOIL

In [8]:
# Chemin du dossier de sortie
output_directory =Config.SOL_EAU_DEK
#if os.path.exists(output_directory):
#    shutil.rmtree(output_directory)
# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data, year, month, dek, days,'volumetric_soil_water_layer_1',area_bf) for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:47:13,304 INFO Welcome to the CDS
2024-07-15 17:47:13,311 INFO Welcome to the CDS
2024-07-15 17:47:13,311 INFO Welcome to the CDS
2024-07-15 17:47:13,311 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:13,317 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:13,318 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:13,602 INFO Request is queued
2024-07-15 17:47:13,634 INFO Request is completed
2024-07-15 17:47:13,634 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data0/adaptor.mars.internal-1720926373.4595983-18780-18-d23feed8-e693-4e58-b7cd-e348d9e58b3b.nc to ..\ASSETS\DATA\RAW\ERA5\SOL_EAU\SOL_EAU_2001_01_03.nc (128.1K)
  0%|          | 0.00/128k [00:00<?, ?B/s]2024-07-15 17:47:13,856 INFO Downloading https://dow

Téléchargement réussi pour 2001-01 03


  7%|▋         | 8.00k/117k [00:00<00:02, 53.5kB/s]2024-07-15 17:47:14,773 INFO Request is running
2024-07-15 17:47:15,083 INFO Download rate 95K/s   


Téléchargement réussi pour 2001-01 02


2024-07-15 17:47:22,584 INFO Request is completed
2024-07-15 17:47:22,584 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data6/adaptor.mars.internal-1721065647.760827-15532-9-28a1939e-b914-47b3-9494-32bcbfce2d08.nc to ..\ASSETS\DATA\RAW\ERA5\SOL_EAU\SOL_EAU_2001_01_01.nc (116.6K)
2024-07-15 17:47:23,988 INFO Download rate 83.5K/s 


Téléchargement réussi pour 2001-01 01
Téléchargement terminé.


### IV EAU PRECIPITABLE

In [9]:
# Chemin du dossier de sortie
output_directory =Config.PREC_EAU_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data, year, month, dek, days,'vertical_integral_of_divergence_of_cloud_liquid_water_flux',area_bf) for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")

#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:47:24,426 INFO Welcome to the CDS
2024-07-15 17:47:24,430 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:24,430 INFO Welcome to the CDS
2024-07-15 17:47:24,430 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:24,430 INFO Welcome to the CDS
2024-07-15 17:47:24,442 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-15 17:47:24,695 INFO Request is queued
2024-07-15 17:47:24,713 INFO Request is completed
2024-07-15 17:47:24,713 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data0/adaptor.mars.internal-1720929235.1198535-25635-8-25b00a12-45fb-4522-bbc8-6a9b285d13d2.nc to ..\ASSETS\DATA\RAW\ERA5\PREC_EAU\PREC_EAU_2001_01_02.nc (116.6K)
2024-07-15 17:47:25,422 INFO Download rate 164.6K/s


Téléchargement réussi pour 2001-01 02


2024-07-15 17:47:25,946 INFO Request is queued
2024-07-15 17:47:25,946 INFO Request is queued
2024-07-15 17:47:27,671 INFO Request is running
2024-07-15 17:47:30,134 INFO Request is queued
2024-07-15 17:47:33,727 INFO Request is running
2024-07-15 17:47:33,727 INFO Request is completed
2024-07-15 17:47:33,727 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data0/adaptor.mars.internal-1721065660.1274512-1585-6-df6dd61c-bdbb-4cc9-ba48-2846a33670c8.nc to ..\ASSETS\DATA\RAW\ERA5\PREC_EAU\PREC_EAU_2001_01_03.nc (128.2K)
2024-07-15 17:47:35,162 INFO Download rate 89.3K/s 


Téléchargement réussi pour 2001-01 03


2024-07-15 17:47:38,944 INFO Request is running
2024-07-15 17:47:46,717 INFO Request is completed
2024-07-15 17:47:46,717 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data3/adaptor.mars.internal-1721065667.7123187-6635-16-d8a4cd49-2c5e-4f80-b0a9-912a969d3fb1.nc to ..\ASSETS\DATA\RAW\ERA5\PREC_EAU\PREC_EAU_2001_01_01.nc (116.6K)
2024-07-15 17:47:48,161 INFO Download rate 81.7K/s 


Téléchargement réussi pour 2001-01 01
Téléchargement terminé.


### V VENT ALTITUDE

#### V.1- WIND V 975 hPa

In [10]:
# Chemin du dossier de sortie
output_directory =Config.V_WIND_975_DEK

os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'v_component_of_wind',area_bf,'975') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()
print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:47:48,572 INFO Welcome to the CDS
2024-07-15 17:47:48,572 INFO Welcome to the CDS
2024-07-15 17:47:48,588 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:47:48,572 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:47:48,572 INFO Welcome to the CDS
2024-07-15 17:47:48,592 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:47:48,889 INFO Request is queued
2024-07-15 17:47:48,920 INFO Request is completed
2024-07-15 17:47:48,920 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data9/adaptor.mars.internal-1720932088.144655-30316-13-6a6be187-2fc9-4789-bfbd-ea1705d8194f.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_975\V_WIND_975_2001_01_02.nc (116.6K)
2024-07-15 17:47:48,935 INFO Downloading https://download-0018.copernicus-climate.e

Téléchargement réussi pour 2001-01 02


 33%|███▎      | 42.0k/128k [00:00<00:00, 133kB/s] 2024-07-15 17:47:50,106 INFO Request is queued
2024-07-15 17:47:50,246 INFO Download rate 97.8K/s


Téléchargement réussi pour 2001-01 03


2024-07-15 17:47:57,776 INFO Request is running
2024-07-15 17:48:10,786 INFO Request is completed
2024-07-15 17:48:10,786 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data2/adaptor.mars.internal-1721065691.9390054-29571-4-3b70a9bd-551f-4198-9190-55fb35e3986c.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_975\V_WIND_975_2001_01_01.nc (116.6K)
2024-07-15 17:48:11,711 INFO Download rate 126.1K/s


Téléchargement réussi pour 2001-01 01
Téléchargement terminé.


#### V.2- WIND V 925 hPa

In [11]:
# Chemin du dossier de sortie
output_directory =Config.V_WIND_925_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'v_component_of_wind',area_bf,'925') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:48:12,118 INFO Welcome to the CDS
2024-07-15 17:48:12,118 INFO Welcome to the CDS
2024-07-15 17:48:12,118 INFO Welcome to the CDS
2024-07-15 17:48:12,118 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:12,134 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:12,140 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:12,392 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data3/adaptor.mars.internal-1720934821.4933712-23644-5-a4e4097f-8d5d-4801-8c82-a67a3f0a8db7.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_925\V_WIND_925_2001_01_01.nc (116.6K)
2024-07-15 17:48:12,421 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data9/adaptor.mars.internal-1720934892.5056057-2651-2-cdce874b-f68c-46

Téléchargement réussi pour 2001-01 03
Téléchargement réussi pour 2001-01 01
Téléchargement réussi pour 2001-01 02
Téléchargement terminé.


#### V.2- WIND V 850 hPa

In [12]:
# Chemin du dossier de sortie
output_directory =Config.V_WIND_850_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'v_component_of_wind',area_bf,'850') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:48:13,453 INFO Welcome to the CDS
2024-07-15 17:48:13,460 INFO Welcome to the CDS
2024-07-15 17:48:13,460 INFO Welcome to the CDS
2024-07-15 17:48:13,467 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:13,467 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:13,460 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:13,751 INFO Request is queued
2024-07-15 17:48:13,831 INFO Request is completed
2024-07-15 17:48:13,831 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data5/adaptor.mars.internal-1720937589.560114-30431-19-9b2961c0-cbce-4498-b5e1-c7528fc26052.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_850\V_WIND_850_2001_01_03.nc (128.1K)
2024-07-15 17:48:13,959 INFO Request is queued
2024-07-15 17:48:15,144 INFO Downloa

Téléchargement réussi pour 2001-01 03


2024-07-15 17:48:35,667 INFO Request is completed
2024-07-15 17:48:35,669 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1721065722.341802-4219-10-0539e7b3-a2d3-4ed0-8350-76464fe3663e.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_850\V_WIND_850_2001_01_02.nc (116.6K)
2024-07-15 17:48:35,796 INFO Request is running
2024-07-15 17:48:37,100 INFO Download rate 81.6K/s 


Téléchargement réussi pour 2001-01 02


2024-07-15 17:48:47,453 INFO Request is completed
2024-07-15 17:48:47,453 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data9/adaptor.mars.internal-1721065728.6603832-32387-19-df0edaaa-abd8-45fd-9c16-123bef245d9d.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_850\V_WIND_850_2001_01_01.nc (116.6K)
2024-07-15 17:48:49,609 INFO Download rate 54.1K/s 


Téléchargement réussi pour 2001-01 01
Téléchargement terminé.


#### V.2- WIND V 700 hPa

In [13]:
# Chemin du dossier de sortie
output_directory =Config.V_WIND_700_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'v_component_of_wind',area_bf,'700') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:48:50,018 INFO Welcome to the CDS
2024-07-15 17:48:50,022 INFO Welcome to the CDS
2024-07-15 17:48:50,022 INFO Welcome to the CDS
2024-07-15 17:48:50,028 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:50,028 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:50,022 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:48:50,290 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data9/adaptor.mars.internal-1720940340.3774824-5686-2-2d38c869-c266-498c-824d-5ad019ab75f0.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_700\V_WIND_700_2001_01_02.nc (116.6K)
2024-07-15 17:48:50,312 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data3/adaptor.mars.internal-1720940359.6337018-16192-1-1e6c7c40-5

Téléchargement réussi pour 2001-01 02
Téléchargement réussi pour 2001-01 01


2024-07-15 17:48:53,195 INFO Request is running
2024-07-15 17:49:12,327 INFO Request is completed
2024-07-15 17:49:12,327 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data3/adaptor.mars.internal-1721065755.3720424-8486-4-1eb5a310-6754-4352-8d76-482235dba087.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_700\V_WIND_700_2001_01_03.nc (128.1K)
2024-07-15 17:49:13,054 INFO Download rate 176.1K/s


Téléchargement réussi pour 2001-01 03
Téléchargement terminé.


#### V.3- WIND V 200 hPa

In [14]:
# Chemin du dossier de sortie
output_directory =Config.V_WIND_200_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'v_component_of_wind',area_bf,'200') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:49:13,457 INFO Welcome to the CDS
2024-07-15 17:49:13,461 INFO Welcome to the CDS
2024-07-15 17:49:13,466 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:13,461 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:13,461 INFO Welcome to the CDS
2024-07-15 17:49:13,476 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:13,772 INFO Request is queued
2024-07-15 17:49:13,788 INFO Request is completed
2024-07-15 17:49:13,788 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data8/adaptor.mars.internal-1720943132.4428618-27875-3-0b621eb0-f369-4acc-be03-b82d3f0f8c29.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_200\V_WIND_200_2001_01_02.nc (116.6K)
 30%|███       | 35.0k/117k [00:00<00:00, 96.8kB/s]2024-07-15 17:49:14,996 INFO Req

Téléchargement réussi pour 2001-01 02


2024-07-15 17:49:16,643 INFO Request is running
2024-07-15 17:49:16,661 INFO Request is queued
2024-07-15 17:49:19,120 INFO Request is running
2024-07-15 17:49:22,854 INFO Request is queued
2024-07-15 17:49:28,108 INFO Request is running
2024-07-15 17:49:28,108 INFO Request is completed
2024-07-15 17:49:28,108 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data3/adaptor.mars.internal-1721065770.622612-18068-2-ea6d70e8-1747-44b1-b6b4-f9303ea09db4.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_200\V_WIND_200_2001_01_01.nc (116.6K)
2024-07-15 17:49:30,550 INFO Download rate 47.7K/s 


Téléchargement réussi pour 2001-01 01


2024-07-15 17:49:35,896 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data6/adaptor.mars.internal-1721065779.1605437-7595-12-baaf7965-9584-4047-85f6-27d0a1e1eda1.nc to ..\ASSETS\DATA\RAW\ERA5\V_WIND_200\V_WIND_200_2001_01_03.nc (128.1K)
2024-07-15 17:49:37,316 INFO Download rate 91K/s   


Téléchargement réussi pour 2001-01 03
Téléchargement terminé.


#### V.4- WIND U 975 hPa

In [15]:
# Chemin du dossier de sortie
output_directory =Config.U_WIND_975_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'u_component_of_wind',area_bf,'975') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:49:37,718 INFO Welcome to the CDS
2024-07-15 17:49:37,736 INFO Welcome to the CDS
2024-07-15 17:49:37,736 INFO Welcome to the CDS
2024-07-15 17:49:37,742 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:37,742 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:37,736 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:38,026 INFO Request is queued
2024-07-15 17:49:38,026 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data1/adaptor.mars.internal-1720946015.870326-26656-11-ea5c741d-9b6e-4452-b281-b5462570f3df.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_975\U_WIND_975_2001_01_02.nc (116.6K)
 35%|███▌      | 41.0k/117k [00:00<00:00, 142kB/s] 2024-07-15 17:49:39,222 INFO Request is running
2024-07-15 17:49:39,238 INFO Download ra

Téléchargement réussi pour 2001-01 02


2024-07-15 17:49:40,886 INFO Request is queued
2024-07-15 17:49:40,919 INFO Request is running
2024-07-15 17:49:43,367 INFO Request is queued
2024-07-15 17:49:46,942 INFO Request is running
2024-07-15 17:49:46,942 INFO Request is completed
2024-07-15 17:49:46,942 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data0/adaptor.mars.internal-1721065791.6872532-1488-7-5c0d7ca9-dd4e-4af5-9ae1-2cd960d46df9.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_975\U_WIND_975_2001_01_01.nc (116.6K)
2024-07-15 17:49:48,412 INFO Download rate 79.3K/s 


Téléchargement réussi pour 2001-01 01


2024-07-15 17:49:52,187 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data6/adaptor.mars.internal-1721065798.8940308-11847-8-d88197bf-2871-40e4-ab00-7d258ef98152.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_975\U_WIND_975_2001_01_03.nc (128.1K)
2024-07-15 17:49:53,551 INFO Download rate 93.9K/s 


Téléchargement réussi pour 2001-01 03
Téléchargement terminé.


#### V.4- WIND U 925 hPa

In [16]:
# Chemin du dossier de sortie
output_directory =Config.U_WIND_925_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'u_component_of_wind',area_bf,'925') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:49:53,991 INFO Welcome to the CDS
2024-07-15 17:49:54,007 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:54,007 INFO Welcome to the CDS
2024-07-15 17:49:54,023 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:53,991 INFO Welcome to the CDS
2024-07-15 17:49:54,023 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:49:54,306 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data2/adaptor.mars.internal-1720948938.3179986-13327-8-6c9f6931-ae97-4b48-b9aa-4aeaaed9bd41.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_925\U_WIND_925_2001_01_03.nc (128.1K)
2024-07-15 17:49:54,368 INFO Request is queued
2024-07-15 17:49:55,173 INFO Download rate 147.8K/s


Téléchargement réussi pour 2001-01 03


2024-07-15 17:49:55,583 INFO Request is running
2024-07-15 17:49:57,171 INFO Request is queued
2024-07-15 17:49:57,223 INFO Request is running
2024-07-15 17:50:32,531 INFO Request is completed
2024-07-15 17:50:32,531 INFO Request is completed
2024-07-15 17:50:32,531 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data0/adaptor.mars.internal-1721065808.1979907-10939-18-3d4372a1-f514-4685-b85b-7a416540ec23.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_925\U_WIND_925_2001_01_02.nc (116.6K)
2024-07-15 17:50:32,531 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data1/adaptor.mars.internal-1721065815.92516-31977-13-f9a67a79-0f36-4241-84d6-a25eb2caafb7.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_925\U_WIND_925_2001_01_01.nc (116.6K)
2024-07-15 17:50:34,343 INFO Download rate 64.9K/s 
2024-07-15 17:50:34,343 INFO Download rate 64.9K/s


Téléchargement réussi pour 2001-01 02Téléchargement réussi pour 2001-01 01

Téléchargement terminé.


#### V.4- WIND U 850 hPa

In [17]:
# Chemin du dossier de sortie
output_directory =Config.U_WIND_850_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'u_component_of_wind',area_bf,'850') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:50:34,758 INFO Welcome to the CDS
2024-07-15 17:50:34,758 INFO Welcome to the CDS
2024-07-15 17:50:34,758 INFO Welcome to the CDS
2024-07-15 17:50:34,758 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:34,762 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:34,762 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:35,091 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data5/adaptor.mars.internal-1720951939.6157389-17273-15-06e65c1d-b4f1-42f0-b929-7accd5f7e506.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_850\U_WIND_850_2001_01_02.nc (116.6K)
2024-07-15 17:50:35,091 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data2/adaptor.mars.internal-1720952008.242049-23581-14-34ab7047

Téléchargement réussi pour 2001-01 01


2024-07-15 17:50:36,082 INFO Download rate 129.4K/s
2024-07-15 17:50:36,213 INFO Download rate 104K/s


Téléchargement réussi pour 2001-01 03
Téléchargement réussi pour 2001-01 02
Téléchargement terminé.


#### V.5- WIND U 700 hPa

In [18]:
# Chemin du dossier de sortie
output_directory =Config.U_WIND_700_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'u_component_of_wind',area_bf,'700') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

2024-07-15 17:50:36,594 INFO Welcome to the CDS
2024-07-15 17:50:36,594 INFO Welcome to the CDS
2024-07-15 17:50:36,594 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:36,594 INFO Welcome to the CDS
2024-07-15 17:50:36,594 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:36,604 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2024-07-15 17:50:36,836 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data5/adaptor.mars.internal-1720954832.4588094-27011-5-00110077-0fc5-43a1-b7e5-0368a1f0af91.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_700\U_WIND_700_2001_01_01.nc (116.6K)
2024-07-15 17:50:36,886 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data7/adaptor.mars.internal-1720954786.1513026-7342-6-32ebbf18-5

Téléchargement réussi pour 2001-01 01
Téléchargement réussi pour 2001-01 02


2024-07-15 17:50:38,076 INFO Request is running
2024-07-15 17:50:45,734 INFO Request is completed
2024-07-15 17:50:45,734 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data1/adaptor.mars.internal-1721065851.6475067-24620-7-318fd657-2687-4466-9a8b-c5c59547432b.nc to ..\ASSETS\DATA\RAW\ERA5\U_WIND_700\U_WIND_700_2001_01_03.nc (128.1K)
2024-07-15 17:50:47,050 INFO Download rate 97.4K/s 


Téléchargement réussi pour 2001-01 03
Téléchargement terminé.


#### V.6- WIND U 200 hPa

In [20]:
# Chemin du dossier de sortie
output_directory =Config.U_WIND_200_DEK

# Créez un nouveau dossier
os.makedirs(output_directory, exist_ok=True)
# Utilisez ThreadPoolExecutor pour télécharger en parallèle
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(download_data_level, year, month, dek, days,'u_component_of_wind',area_bf,'200') for year, month, dek, days in tasks]
    for future in as_completed(futures):
        future.result()

print("Téléchargement terminé.")
#download_data(year, month, dek, days,parametre,area=area_bf,level=None)

Téléchargement terminé.
